In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from __future__ import division             # Division in Python 2.7
import matplotlib
matplotlib.use('Agg')                       # So that we can render files without GUI
import matplotlib.pyplot as plt
from matplotlib import rc
import numpy as np
import math as m

from matplotlib import colors

def plot_color_gradients(gradients, names):
    # For pretty latex fonts (commented out, because it does not work on some machines)
    #rc('text', usetex=True) 
    #rc('font', family='serif', serif=['Times'], size=10)
    rc('legend', fontsize=10)

    column_width_pt = 400         # Show in latex using \the\linewidth
    pt_per_inch = 72
    size = column_width_pt / pt_per_inch

    fig, axes = plt.subplots(nrows=len(gradients), sharex=True, figsize=(size, 0.75 * size))
    fig.subplots_adjust(top=1.00, bottom=0.05, left=0.25, right=0.95)


    for ax, gradient, name in zip(axes, gradients, names):
        # Create image with two lines and draw gradient on it
        img = np.zeros((2, 1024, 3))
        for i, v in enumerate(np.linspace(0, 1, 1024)):
            img[:, i] = gradient(v)

        im = ax.imshow(img, aspect='auto')
        im.set_extent([0, 1, 0, 1])
        ax.yaxis.set_visible(False)

        pos = list(ax.get_position().bounds)
        x_text = pos[0] - 0.25
        y_text = pos[1] + pos[3]/2.
        fig.text(x_text, y_text, name, va='center', ha='left', fontsize=10)

    fig.savefig('my-gradients.pdf')


In [2]:
# Liczba próbek w gradiencie
samples = 1024
# Lista punktów tworzących odcinki na stożku
hsvGBR = [[120,1,1],[180,1,1],[240,1,1],[300,1,1],[360,1,1]]
hsvUnknown = [[0,0.7,0.9],[180,0.7,0.9],[360,0.7,0.9]]
hsvCustom = [[0,1,0],[60,0.8,0.2],[120,0.6,0.4],[180,0.4,0.6],[240,0.2,0.8],[300,0,1]]

def getcolor(colors, v):
    x= (v * (len(colors) - 1))
    color = m.trunc(x)
    if color == len(colors) - 1:
        color = len(colors) - 2
    scale = x - color
    return color, scale

def grad_RGB(colors,v):
    color, scale = getcolor(colors, v)
    fpoint = []
    for i in range(len(colors[color])):
        L = colors[color][i]
        P = colors[color + 1][i]
        div = P - L
        if div != 0:
            fpoint.append(L + (1 / (div)) * scale)
        else:
            fpoint.append(L)
    return fpoint

def grad_HSV(colors,v):
    color,scale = getcolor(colors,v)
    fpoint = []
    for i in range(len(colors[color])):
        L = colors[color][i] 
        P = colors[color + 1][i]
        div = P - L
        fpoint.append(L + div * scale)
    return fpoint

In [3]:
def hsv2rgb(h, s, v):
    vs = v*s
    if h > 0:
        while h > 360:
            h -= 360
    else:
        while h < 0:
            h += 360
    hue = h/60
    x = vs * (1 - abs((hue % 2) -1))
    switch = {
        0: [vs,x,0],
        1: [x,vs,0],
        2: [0,vs,x],
        3: [0,x,vs],
        4: [x,0,vs],
        5: [vs,0,x],
        6: [vs,x,0]
    }
    rgb = switch.get(m.trunc(hue),[0,0,0])
    match = v-vs
    rgb = [i+match for i in rgb]
    return rgb

In [4]:
def gradient_rgb_bw(v):
    return grad_RGB([[0,0,0],[1,1,1]],v)

In [5]:
def gradient_rgb_gbr(v):
    return grad_RGB([[0,1,0],[0,0,1],[1,0,0]],v)

In [6]:
def gradient_rgb_gbr_full(v):
    return grad_RGB([[0,1,0],[0,1,1],[0,0,1],[1,0,1],[1,0,0]],v)

In [7]:
def gradient_rgb_wb_custom(v):
    return grad_RGB([[0,0,0],[0,0,1],[1,0,1],[1,0,0],[1,1,0],[0,1,0],[0,1,1],[1,1,1]],v)

In [8]:
def gradient_hsv_bw(v):
    hsv = grad_HSV([[0,0,0],[0,0,1]],v)
    return hsv2rgb(hsv[0], hsv[1], hsv[2])

In [9]:
def gradient_hsv_gbr(v):
    hsv = grad_HSV([[120,1,1],[180,1,1],[240,1,1],[300,1,1],[360,1,1]],v)
    return hsv2rgb(hsv[0], hsv[1], hsv[2])

In [10]:
def gradient_hsv_unknown(v):
    hsv = grad_HSV([[0,0.7,0.9],[180,0.7,0.9],[360,0.7,0.9]],v)
    return hsv2rgb(hsv[0], hsv[1], hsv[2])

In [11]:
def gradient_hsv_custom(v):
    hsv = grad_HSV([[0,1,0],[60,0.8,0.2],[120,0.6,0.4],[180,0.4,0.6],[240,0.2,0.8],[300,0,1]],v)
    return hsv2rgb(hsv[0], hsv[1], hsv[2])

In [12]:
if __name__ == '__main__':
    def toname(g):
        return g.__name__.replace('gradient_', '').replace('_', '-').upper()

    gradients = (gradient_rgb_bw, gradient_rgb_gbr, gradient_rgb_gbr_full, gradient_rgb_wb_custom,
                 gradient_hsv_bw, gradient_hsv_gbr, gradient_hsv_unknown, gradient_hsv_custom)

    plot_color_gradients(gradients, [toname(g) for g in gradients])